## Description：
这个是参考了一个GitHub纯手撸了一遍， 通过这个能够加深对FFM内部的理解， 具体内容可以参考[https://github.com/Orisun/ffm](https://github.com/Orisun/ffm).<br><br>
由于FFM对于数据集的存储格式有特殊的要求， 所以这里用的自己生成的训练集和测试集。 数据格式: label field_name, feature_name, value。

## 导入包

In [1]:
import numpy as np
import math

## 定义Singleton（单例模式)
这个继承type， 继承type的类在python的概念中叫做元类，在python中所有的对象都是类，包括类本身。元类就是创建类的类。 <br><br>type和object类的关系: type是所有类的元类（包括object），object是所有类的父类（包括type）

In [8]:
class Singleton(type):
    def __init__(cls, class_name, base_classes, attr_dict):
        cls.__instance = None
        super(Singleton, cls).__init__(class_name, base_classes, attr_dict)
    
    def __call__(cls, *args, **kwargs):
        if cls.__instance is None:
            cls.__instance = super(Singleton, cls).__call__(*args, **kwargs)
        else:
            return cls.__instance

## 定义逻辑回归类

In [9]:
class Logistic(object):
    __metaclass__ = Singleton   # 单例
    
    def __init__(self):
        self.exp_max = 10.0
        self.exp_scale = 0.001
        self.exp_intv = int(self.exp_max / self.exp_scale)   # 10000
        self.exp_table = [0.0] * self.exp_intv      # 10000个0
        for i in range(self.exp_intv):   # [10000]
            x = self.exp_scale * i # [0.001 * i for i in range(0, 10000)]   x 是0-10
            exp = math.exp(x)  # 1-e^10
            self.exp_table[i] = exp / (1.0 + exp)       # 归一化缩放  [0, 1]     
    
    # 这里是手动求逻辑回归函数的值
    def decide_by_table(self, x):
        """查表获得逻辑回归的函数值"""
        if x == 0:
            return 0.5
        i = int(np.nan_to_num(abs(x) / self.exp_scale))
        y = self.exp_table[min(i, self.exp_intv - 1)]
        if x > 0:
            return y
        else:
            return 1.0 - y
    
    def decide_by_tanh(self, x):
        '''直接使用1.0 / (1.0 + np.exp(-x))容易发警告“RuntimeWarning: overflowencountered in exp”，
           转换成如下等价形式后算法会更稳定
        '''
        return 0.5 * (1 + np.tanh(0.5 * x))

    def decide(self, x):
        """原始的sigmoid函数"""
        return 1.0 / (1.0 + np.exp(-x))

## 定义FFM模型

In [16]:
class FFM_Node(object):
    '''
    通常x是高维稀疏向量，所以用链表来表示一个x，链表上的每个节点是个3元组(j,f,v)
    '''
    __slots__ = ['j', 'f', 'v']    # 按照元组不是字典的方式存储类的成员属性
    
    def __init__(self, j, f, v):
        """
            j: Feature index (0-n-1)
            f: field index(0-m-1)
            v: value
        """
        self.j = j
        self.f = f
        self.v = v
    
class FFM(object):
    def __init__(self, m, n, k, eta, lambd):
        """
            m: Number of fields
            n: Number of features
            k: Number of latent factors
            eta: learning rate
            lambd: regularization coefficient
        """
        self.m = m
        self.n = n
        self.k = k
        
        #超参数
        self.eta = eta
        self.lambd = lambd
        
        # 初始化三维权重矩阵w~U(0, 1/sqrt(k))
        self.w = np.random.rand(n, m, k) / math.sqrt(k)
        
        # 初始化累积梯度平方和， AdaGrad时要用到
        self.G = np.ones(shape=(n, m, k), dtype=np.float64)
        self.log = Logistic()
    
    # 这个是计算第三项
    def phi(self, node_list):
        """
        特征组合式的线性加权求和
        param node_list: 用链表存储x中的非0值
        """
        z = 0.0
        for a in range(len(node_list)):
            node1 = node_list[a]
            j1 = node1.j
            f1 = node1.f
            v1 = node1.v
            for b in range(a+1, len(node_list)):
                node2 = node_list[b]
                j2 = node2.j
                f2 = node2.f
                v2 = node2.v
                w1 = self.w[j1, f2]
                w2 = self.w[j2, f1]
                z += np.dot(w1, w2) * v1 * v2
        
        return z
    
    
    def predict(self, node_list):
        """
        输入x， 预测y的值
        """
        z = self.phi(node_list)
        y = self.log.decide_by_tanh(z)
        return y

    # 随机梯度下降
    def sgd(self, node_list, y):
        """
        根据一个样本更新模型参数：
        node_list: 链表存储x中的非0值
        y: 正样本1， 负样本-1
        """
        kappa = -y / (1+math.exp(y*self.phi(node_list)))    # 论文里面的那个导数
        for a in range(len(node_list)):
            node1 = node_list[a]
            j1 = node1.j
            f1 = node1.f
            v1 = node1.v
            for b in range(a+1, len(node_list)):
                node2 = node_list[b]
                j2 = node2.j
                f2 = node2.f
                v2 = node2.v
                c = kappa * v1 * v2      # 这是求导数
                
                # self.w[j1,f2]和self.w[j2,f1]是向量，导致g_j1_f2和g_j2_f1也是向量
                g_j1_f2 = self.lambd * self.w[j1, f2] + c * self.w[j2, f1]
                g_j2_f1 = self.lambd * self.w[j2, f1] + c * self.w[j1, f2]
                
                # 计算各个维度上的梯度累积平方和
                self.G[j1, f2] += g_j1_f2 ** 2
                self.G[j2, f1] += g_j2_f1 ** 2
                
                # Adagrad 算法
                self.w[j1, f2] -= self.eta / np.sqrt(self.G[j1, f2]) * g_j1_f2  # sqrt(G)作为分母，所以G必须是大于0的正数
                self.w[j2, f1] -= self.eta / np.sqrt(
                    self.G[j2, f1]) * g_j2_f1  # math.sqrt()只能接收一个数字作为参数，而numpy.sqrt()可以接收一个array作为参数，表示对array中的每个元素分别开方
    
    # 训练
    def train(self, sample_generator, max_echo, max_r2):
        """
        根据一堆样本训练模型
        sample_generator: 样本生成器，每次yield (node_list, y)，node_list中存储的是x的非0值。通常x要事先做好归一化，即模长为1，这样精度会略微高一点
        max_echo: 最大迭代次数
        max_r2: 拟合系数r2达到阈值时即可终止学习
        """
        for itr in range(max_echo):
            print("echo: ", itr)
            y_sum = 0.0
            y_sqare_sum = 0.0
            err_square_sum = 0.0    # 误差平方和
            population = 0   # 样本总数
            for node_list, y in sample_generator:
                y = 0.0  if y == -1 else y    # 真实的y取值为{-1,1}，而预测的y位于(0,1)，计算拟合效果时需要进行统一
                self.sgd(node_list, y)
                y_hat = self.predict(node_list)
                y_sum += y
                y_sqare_sum += y ** 2
                err_square_sum += (y-y_hat) ** 2
                population += 1
            
            var_y = y_sqare_sum - y_sum * y_sum / population  # y的方差
            r2 = 1 - err_square_sum / var_y
            print("r2: ", r2)
            if r2 > max_r2:
                print("r2 have reach", r2)
                break
        
    # 模型保存
    def save_model(self, outfile):
        '''
        序列化模型
        :param outfile:
        :return:
        '''
        np.save(outfile, self.w)

    def load_model(self, infile):
        '''
        加载模型
        :param infile:
        :return:
        '''
        self.w = np.load(infile)

## 模型测试

In [11]:
import re
class Sample(object):
    def __init__(self, infile):
        self.infile = infile
        self.regex = re.compile("\\s+")

    def __iter__(self):
        with open(self.infile, 'r') as f_in:
            for line in f_in:
                arr = self.regex.split(line.strip())
                if len(arr) >= 2:
                    y = float(arr[0])
                    assert math.fabs(y) == 1
                    node_list = []
                    square_sum = 0.0
                    for i in range(1, len(arr)):
                        brr = arr[i].split(",")
                        if len(brr) == 3:
                            j = int(brr[0])
                            f = int(brr[1])
                            v = float(brr[2])
                            square_sum += v * v
                            node_list.append(FFM_Node(j, f, v))
                    if square_sum > 0:
                        norm = math.sqrt(square_sum)
                        # 把模长缩放到1
                        normed_node_list = [FFM_Node(ele.j, ele.f, ele.v / norm) for ele in node_list]
                        yield (normed_node_list, y)

In [17]:
# 设置参数   5个特征， 2个域， 2维的k
n = 5
m = 2
k = 2

# 路径
train_file = "dataset/train.txt"
valid_file = "dataset/test.txt"
model_file = "ffm.npy"

# 超参数
eta = 0.01
lambd = 1e-2
max_echo = 30
max_r2 = 0.9

# 训练模型，并保存模型参数
sample_generator = Sample(train_file)
ffm = FFM(m, n, k, eta, lambd)
ffm.train(sample_generator, max_echo, max_r2)
ffm.save_model(model_file)

echo:  0
r2:  -0.16471447551138296
echo:  1
r2:  -0.16470221265226326
echo:  2
r2:  -0.16468995371475814
echo:  3
r2:  -0.1646776986975249
echo:  4
r2:  -0.1646654475992202
echo:  5
r2:  -0.1646532004185024
echo:  6
r2:  -0.16464095715402927
echo:  7
r2:  -0.1646287178044601
echo:  8
r2:  -0.16461648236845394
echo:  9
r2:  -0.16460425084467079
echo:  10
r2:  -0.1645920232317708
echo:  11
r2:  -0.16457979952841506
echo:  12
r2:  -0.16456757973326486
echo:  13
r2:  -0.16455536384498193
echo:  14
r2:  -0.1645431518622289
echo:  15
r2:  -0.16453094378366884
echo:  16
r2:  -0.16451873960796504
echo:  17
r2:  -0.16450653933378123
echo:  18
r2:  -0.16449434295978183
echo:  19
r2:  -0.1644821504846321
echo:  20
r2:  -0.16446996190699736
echo:  21
r2:  -0.16445777722554378
echo:  22
r2:  -0.16444559643893775
echo:  23
r2:  -0.1644334195458459
echo:  24
r2:  -0.16442124654493617
echo:  25
r2:  -0.16440907743487654
echo:  26
r2:  -0.1643969122143356
echo:  27
r2:  -0.164384750881982
echo:  28
r2:

In [18]:
# 加载模型，并计算在验证集上的拟合效果
ffm.load_model(model_file)
valid_generator = Sample(valid_file)
y_sum = 0.0
y_square_sum = 0.0
err_square_sum = 0.0  # 误差平方和
population = 0  # 样本总数
for node_list, y in valid_generator:
    y = 0.0 if y == -1 else y  # 真实的y取值为{-1,1}，而预测的y位于(0,1)，计算拟合效果时需要进行统一
    y_hat = ffm.predict(node_list)
    y_sum += y
    y_square_sum += y ** 2
    err_square_sum += (y - y_hat) ** 2
    population += 1
var_y = y_square_sum - y_sum * y_sum / population  # y的方差
r2 = 1 - err_square_sum / var_y

In [19]:
r2

-0.027269624863842212